In [1]:
import pandas as pd
from pathlib import Path

# ==============================================================================
# 第一步：定义文件路径
# ==============================================================================
# 使用您指定的输入文件路径
input_dir = Path('../../Data/0')
input_file = input_dir / '处理后的数据.xlsx'

# 使用您指定的输出目录路径
output_dir = Path('../../Data/1')
# 确保输出目录存在，如果不存在则自动创建
output_dir.mkdir(parents=True, exist_ok=True)

print(f"--- 数据拆分任务启动 ---")
print(f"输入文件: {input_file}")
print(f"输出目录: {output_dir}")

# ==============================================================================
# 第二步：加载预处理好的总表
# ==============================================================================
try:
    # 我们只关心已分类的数据表，因为未分类的没有风化信息
    df_cleaned = pd.read_excel(input_file, sheet_name='已分类清洗后数据')
    print(f"\n成功加载总表，共 {len(df_cleaned)} 条数据，准备开始拆分...")
except FileNotFoundError:
    print(f"❌ 错误：输入文件未找到，请确认路径 {input_file} 是否正确。")
    # 如果文件找不到，则停止执行
    exit()
except Exception as e:
    print(f"❌ 错误：读取文件时发生错误: {e}")
    exit()

# ==============================================================================
# 第三步：执行拆分逻辑
# ==============================================================================
# --- (3.1) 拆分出 表1：纯净未风化样本 ---
# 逻辑：直接筛选出 `表面风化` 为 "无风化" 的所有样本
df_table1 = df_cleaned[df_cleaned['表面风化'] == '无风化'].copy()
df_table1.reset_index(drop=True, inplace=True)


# --- (3.2) 拆分出 表2 和 表3 ---
# 逻辑：首先筛选出所有来自“风化”文物的样本，然后再根据采样点名称进行细分
df_weathered_all_points = df_cleaned[df_cleaned['表面风化'] == '风化'].copy()

# 添加一个辅助列'点类型'，用于区分风化点和未风化点
# 这个逻辑基于我们之前根据题目规则的严格论证
df_weathered_all_points['点类型'] = df_weathered_all_points['文物采样点'].apply(lambda x: '未风化' if '未风化' in str(x) else '风化')

# 拆分出 表2：风化文物上的风化采样点
df_table2 = df_weathered_all_points[df_weathered_all_points['点类型'] == '风化'].copy()
df_table2.reset_index(drop=True, inplace=True)


# 拆分出 表3：风化文物上的未风化采样点
df_table3 = df_weathered_all_points[df_weathered_all_points['点类型'] == '未风化'].copy()
df_table3.reset_index(drop=True, inplace=True)


print("\n✅ 数据拆分逻辑执行完毕。")
print(f"  - 表1 (纯净未风化样本): 共 {len(df_table1)} 条")
print(f"  - 表2 (风化文物的风化点): 共 {len(df_table2)} 条")
print(f"  - 表3 (风化文物的未风化点): 共 {len(df_table3)} 条")
print(f"  - 校验: {len(df_table1)} + {len(df_table2)} + {len(df_table3)} = {len(df_cleaned)} (总数一致)")

# ==============================================================================
# 第四步：保存新的数据表格
# ==============================================================================
try:
    # 定义输出文件名
    file1_path = output_dir / '表1_纯净未风化样本.xlsx'
    file2_path = output_dir / '表2_风化文物的风化点.xlsx'
    file3_path = output_dir / '表3_风化文物的未风化点.xlsx'

    # 将三个新的DataFrame分别保存为Excel文件
    df_table1.to_excel(file1_path, index=False)
    df_table2.to_excel(file2_path, index=False)
    df_table3.to_excel(file3_path, index=False)

    print("\n✅ 所有新的数据表格已成功保存至指定目录。")
    print(f"   -> {file1_path}")
    print(f"   -> {file2_path}")
    print(f"   -> {file3_path}")

except Exception as e:
    print(f"❌ 错误：保存文件时发生错误: {e}")

--- 数据拆分任务启动 ---
输入文件: ..\..\Data\0\处理后的数据.xlsx
输出目录: ..\..\Data\1

成功加载总表，共 67 条数据，准备开始拆分...

✅ 数据拆分逻辑执行完毕。
  - 表1 (纯净未风化样本): 共 25 条
  - 表2 (风化文物的风化点): 共 32 条
  - 表3 (风化文物的未风化点): 共 10 条
  - 校验: 25 + 32 + 10 = 67 (总数一致)

✅ 所有新的数据表格已成功保存至指定目录。
   -> ..\..\Data\1\表1_纯净未风化样本.xlsx
   -> ..\..\Data\1\表2_风化文物的风化点.xlsx
   -> ..\..\Data\1\表3_风化文物的未风化点.xlsx


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# ==============================================================================
# 第一步：加载问题一专属数据
# ==============================================================================
# 定义文件路径
input_dir = Path('../../Data/1')
# 表2是模型的输入(X)
table2_path = input_dir / '表2_风化文物的风化点.xlsx'
# 表3是模型的标准答案(y)
table3_path = input_dir / '表3_风化文物的未风化点.xlsx'

try:
    df_weathered_points = pd.read_excel(table2_path)
    df_unweathered_points = pd.read_excel(table3_path)
    print("--- 成功加载用于问题一预测模型的专属数据表 (表2 和 表3) ---")
except FileNotFoundError as e:
    print(f"❌ 错误：文件未找到。请确认 {e.filename} 是否存在于 {input_dir} 目录中。")
    exit()
except Exception as e:
    print(f"❌ 错误：读取文件时发生错误: {e}")
    exit()

# ==============================================================================
# 第二步：构建成对的回归训练集 (X 和 y)
# ==============================================================================
# 筛选出化学成分列
chemical_columns = [col for col in df_weathered_points.columns if '(' in col and ')' in col]

# 为了确保样本一一对应，我们使用'文物编号'作为索引进行合并
df_weathered_indexed = df_weathered_points.set_index('文物编号')
df_unweathered_indexed = df_unweathered_points.set_index('文物编号')

# 将两个表横向合并，形成一个包含“风化后”和“风化前”成分的宽表
paired_df = pd.merge(
    df_weathered_indexed[chemical_columns],
    df_unweathered_indexed[chemical_columns],
    left_index=True,
    right_index=True,
    suffixes=('_风化后', '_风化前')
)

# 准备特征(X)：所有风化后的化学成分
features_X = [col for col in paired_df.columns if '_风化后' in col]
X_regression = paired_df[features_X]

# 准备目标(y)：这是一个包含了所有风化前成分的DataFrame
targets_y_all = [col for col in paired_df.columns if '_风化前' in col]
y_regression_all = paired_df[targets_y_all]

print(f"✅ 成功构建回归模型的训练数据，共 {len(paired_df)} 对样本。")

# ==============================================================================
# 第三步：为每种化学成分训练并评估一个预测模型
# ==============================================================================
# 创建一个字典，用于存储我们为每种成分训练好的最终模型
trained_regression_models = {}
print("\n--- 开始为每种化学成分训练并评估随机森林回归模型 ---")

# 循环为每种化学成分训练一个模型
for chemical in chemical_columns:
    current_target_col = chemical + '_风化前'
    y_current = y_regression_all[current_target_col]
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    # 使用交叉验证评估模型性能
    r2_scores = cross_val_score(model, X_regression, y_current, cv=5, scoring='r2')
    mae_scores = cross_val_score(model, X_regression, y_current, cv=5, scoring='neg_mean_absolute_error')
    
    # 打印简化的评估结果
    # print(f"  - [{chemical}] R²均值: {np.mean(r2_scores):.4f}, MAE均值: {-np.mean(mae_scores):.4f}")
    
    # 使用所有成对数据训练最终模型，并保存
    model.fit(X_regression, y_current)
    trained_regression_models[chemical] = model

print("✅ 所有化学成分的预测模型均已训练并评估完毕。")


# ==============================================================================
# 第四步：应用模型，生成包含预测结果的表格
# ==============================================================================
print("\n--- 正在应用已训练好的模型进行预测 ---")

# 我们要对`表2`中的所有风化点进行预测
# 首先准备好这些样本的特征数据（确保列名与训练时一致）
X_to_predict = df_weathered_points[chemical_columns]
X_to_predict.columns = [col + '_风化后' for col in X_to_predict.columns]

# 创建一个新的DataFrame来存储预测结果，先放入原始风化后的数据
df_predictions = df_weathered_points[['文物编号', '文物采样点']].copy()
df_predictions = pd.concat([df_predictions, X_to_predict.reset_index(drop=True)], axis=1)


# 循环使用我们训练好的模型，对每个成分进行预测
for chemical, model in trained_regression_models.items():
    # 使用对应成分的模型进行预测
    predicted_values = model.predict(X_to_predict)
    # 将预测结果作为新列添加到结果表格中
    df_predictions[chemical + '_预测风化前'] = predicted_values

print("✅ 预测完成！")

# ==============================================================================
# 第五步：显示并保存最终的处理后表格
# ==============================================================================
# 为了方便在Notebook中直接查看，我们调整一下列的顺序，把对应的成分放在一起
display_order = ['文物编号', '文物采样点']
for chemical in chemical_columns:
    display_order.append(chemical + '_风化后')
    display_order.append(chemical + '_预测风化前')

df_predictions_display = df_predictions[display_order]

print("\n--- 问题一模型处理后的最终表格 (部分预览) ---")
# 使用 .style.format 来美化输出，保留两位小数
# .set_table_styles 使表格更易读
styled_df = df_predictions_display.head().style.format(precision=2).set_table_styles(
    [{'selector': 'th', 'props': [('background-color', '#0c5277'), ('color', 'white'), ('text-align', 'center')]},
     {'selector': 'td', 'props': [('text-align', 'center')]}]
)
# 在Notebook中显示美化后的表格
from IPython.display import display
display(styled_df)


# 保存完整的预测结果表格到新文件
output_prediction_file = input_dir / '问题一预测结果对比表.xlsx'
df_predictions_display.to_excel(output_prediction_file, index=False)

print(f"\n✅ 包含预测结果的完整表格已成功保存至 -> 【{output_prediction_file}】")

--- 成功加载用于问题一预测模型的专属数据表 (表2 和 表3) ---
✅ 成功构建回归模型的训练数据，共 2 对样本。

--- 开始为每种化学成分训练并评估随机森林回归模型 ---


ValueError: Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=2.